In [1]:
import os
from smolagents import tool, CodeAgent, LiteLLMModel
model = LiteLLMModel(model_id='gpt-4o', 
                     api_key=os.getenv("OPENAI_API_KEY"), 
                     timeout=60, # I'm still tier 1 ㅠ_ㅠ
                    )
import datetime
today = datetime.datetime.today().strftime('%Y%m%d')

import sqlite3
conn = sqlite3.connect("Chinook.db")

In [2]:
@tool
def run_sql(query: str) -> list:
    """Executes an SQL query on the global database connection.

    Args:
        query: A valid SQL query.

    Returns:
        list: Query results as a list of tuples, or an error message as a string.
    """
    try:
        results = conn.execute(query).fetchall()
        return results  # Returns a list of tuples
    except Exception as e:
        return [f"Query failed with error: {e}"]  # Returns error as a list with one string element

@tool
def save_code(fpath: str, code: str) -> str:
    """Saves a piece of code at the specified file path.
    
    Args:
        fpath: Path and filename to save the code.
        code: Code snippet to save.
    
    Returns:
        str: Success message or error.
    """
    try:
        with open(fpath, 'w') as f:
            f.write(code)
        return f"Code successfully saved to {fpath}"
    except Exception as e:
        return f"Failed to save code: {e}"


agent = CodeAgent(
    model=model,
    tools=[run_sql, save_code],
    additional_authorized_imports=["matplotlib"],
    max_steps=20,
)

save_path = f'./output_{today}'
if not os.path.exists(save_path):
    os.makedirs(save_path)

append_sys_prompt = f"""\n\nMake sure to save all codes you wrote under '{save_path}'. When you are finished \
with the task and generated all the necessary response and there is nothing else left for final_answer, \
then simply return 'Task completed'."""

agent.prompt_templates['system_prompt'] = agent.prompt_templates['system_prompt'] + append_sys_prompt

In [3]:
task = "Find the total sales amount for each music genre and create a bar chart and save it as 'saves_vs_genre.png'"

In [4]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the total sales amount for each music genre and create a bar chart and save it as 'saves_vs_genre.png'     │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT Genre.Name AS Genre, SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalSales                      
  FROM InvoiceLine                                                                                                 
  JOIN Track ON InvoiceLine.TrackId = Track.TrackId                                                                
  JOIN Genre ON Track.GenreId = Genre.GenreId                                                                      
  GROUP BY Genre.Name                                                                                              
  ORDER BY TotalSales DESC                                                                                         
  """                                                                                                              
  sales_data = run_sql(query=query)                                                                                
  print(sales_data)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
['Query failed with error: no such table: InvoiceLine']

Out: None

[Step 0: Duration 3.69 seconds| Input tokens: 2,145 | Output tokens: 208]

...

[Step 6: Duration 27.39 seconds| Input tokens: 23,730 | Output tokens: 1,642]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Task completed successfully. The bar chart 'saves_vs_genre.png' has been created and the code has  
  been saved.")                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Task completed successfully. The bar chart 'saves_vs_genre.png' has been created and the code 
has been saved.

[Step 7: Duration 33.08 seconds| Input tokens: 29,318 | Output tokens: 1,753]